In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Load saved list for churn and loyal sample users: churn_sample_list, loyal_sample_list
churn_list = np.load('/Users/ZhijingYe/Desktop/data/output/churn_sample_list.npy').tolist()
loyal_list = np.load('/Users/ZhijingYe/Desktop/data/output/loyal_sample_list.npy').tolist()


In [3]:
len(churn_list), len(loyal_list)

(59846, 24886)

In [4]:
set(churn_list) & set(loyal_list)

set()

In [5]:
len(set(churn_list) - set(loyal_list))

59846

### Read in the other columns line by line by matching uid with user_set

In [6]:
import glob

In [7]:
filepath = '/Users/ZhijingYe/Desktop/data/play/*play.log'
files = glob.glob(filepath)
# amount of files
len(files)

138

In [8]:
schema = ['uid','device','song_id','song_type','song_name','singer','play_time','song_length','paid_flag','file_name','label']

In [9]:
output = open('/Users/ZhijingYe/Desktop/data/output/user_sample_play.log','a')

In [10]:
churn_set = set(churn_list)
loyal_set = set(loyal_list)

In [11]:
import time


for the_file in files:
    current_time = time.clock()

    with open(the_file, 'r') as f:
        lines = f.readlines()
        file_name = f.name.split('/')[-1]
        print('processing file: %s' % file_name)
        for line in lines:
            user_id = line.split('\t')[0]
            if user_id in churn_set:
                contents_to_wirte = line.strip('\n').split('\t')
                contents_to_wirte.extend((file_name, '1'))
            elif user_id in loyal_set:
                contents_to_wirte = line.strip('\n').split('\t')
                contents_to_wirte.extend((file_name, '0'))
            else:
                continue  
            output.write('\t'.join(contents_to_wirte)+'\n')
    print('...costs %.2f seconds' % (time.clock()-current_time))

processing file: 20170410_2_play.log
...costs 1.45 seconds
processing file: 20170410_3_play.log
...costs 1.54 seconds
processing file: 20170427_3_play.log
...costs 1.64 seconds
processing file: 20170427_2_play.log
...costs 1.22 seconds
processing file: 20170504_3_play.log
...costs 1.15 seconds
processing file: 20170504_2_play.log
...costs 1.12 seconds
processing file: 20170508_1_play.log
...costs 1.19 seconds
processing file: 20170505_1_play.log
...costs 1.14 seconds
processing file: 20170411_1_play.log
...costs 1.51 seconds
processing file: 20170426_1_play.log
...costs 1.25 seconds
processing file: 20170509_3_play.log
...costs 1.06 seconds
processing file: 20170509_2_play.log
...costs 1.08 seconds
processing file: 20170401_2_play.log
...costs 2.43 seconds
processing file: 20170401_3_play.log
...costs 2.74 seconds
processing file: 20170423_1_play.log
...costs 1.46 seconds
processing file: 20170414_1_play.log
...costs 1.03 seconds
processing file: 20170418_2_play.log
...costs 1.30 secon

In [12]:
output.close()

In [13]:
df_play = pd.read_csv('/Users/ZhijingYe/Desktop/data/output/user_sample_play.log',
                 delimiter='\t',header=None,index_col=None,names = schema,
                 dtype = {'uid':'str', 'song_id':'str','song_type' : 'str'})
df_play.head()

/Users/ZhijingYe/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,uid,device,song_id,song_type,song_name,singer,play_time,song_length,paid_flag,file_name,label
0,168308107,ar,162455,0,最初的梦想,范玮琪,296,296,0,20170410_2_play.log,0
1,168112765,ar,4393501,0,喜欢你(f101 粤),Beyond,272,0,0,20170410_2_play.log,0
2,168274411,ar,22833011,0,宽恕,宽恕乐队,24,156,0,20170410_2_play.log,0
3,0,ar,4266814,1,天使的翅膀,徐誉滕,214384,0,0,20170410_2_play.log,0
4,168274411,ar,176292,0,爱不爱我,零点乐队,333,334,0,20170410_2_play.log,0


In [14]:
df_play.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39701302 entries, 0 to 39701301
Data columns (total 11 columns):
uid            object
device         object
song_id        object
song_type      object
song_name      object
singer         object
play_time      object
song_length    object
paid_flag      object
file_name      object
label          object
dtypes: object(11)
memory usage: 3.5+ GB


In [15]:
user_set = set(df_play['uid'])

In [16]:
len(user_set)

150884

In [17]:
len(user_set & churn_set)-len(churn_set)

-30

In [18]:
len(user_set & loyal_set)-len(loyal_set)

0

In [19]:
user_set - user_set.union(loyal_set)

set()

In [20]:
df_play.isnull().sum(axis = 0)

uid              9426
device              0
song_id          7510
song_type       23109
song_name        5524
singer          49189
play_time      160232
song_length     20717
paid_flag       51979
file_name       51980
label           51980
dtype: int64

### Missing values


In [21]:
# Check those logs with missing uid
df_play[df_play['uid'].isnull()]

,uid,device,song_id,song_type,song_name,singer,play_time,song_length,paid_flag,file_name,label
3164,NaN,朴树,2,23,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN
5618,NaN,刘涛,31,30,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN
5690,NaN,刘涛,51,51,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN
6643,NaN,薛之谦,19,41,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN
7879,NaN,EXO,26,26,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN
11400,NaN,佛教音乐,51,51,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN
11918,NaN,佛教音乐,51,51,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN
19485,NaN,杨洋,0,29,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN
21698,NaN,左宏元&amp;张慧清,0,46,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN
25202,NaN,好想好想(51秒铃声版),0,52,0,20170410_2_play.log,0,NaN,NaN,NaN,NaN


In [22]:
# Looks suspecious as there are lots of missing inputs in these logs, delete these logs
df_play = df_play.loc[df_play.uid.notnull()]


In [23]:
df_play.isnull().sum(axis = 0)

uid                 0
device              0
song_id          7510
song_type       23109
song_name        5524
singer          49164
play_time      160207
song_length     11291
paid_flag       42553
file_name       42554
label           42554
dtype: int64

In [24]:
df_play.song_type.value_counts()

0             29022270
1              6836180
0              2932205
2               474861
1               325546
2                32711
                  9908
3                 4658
刚好遇见你              129
没有你陪伴真的好孤单          93
暧昧                  76
逆流成河                67
演员                  62
3                   55
走着走着就散了             52
...
扎西哥哥(DJ版)                        1
女孩你知道吗                           1
爱的魔法                             1
拥抱你                              1
太阳雨                              1
克罗地亚狂想曲                          1
情翼                               1
腐草为萤                             1
Can You Feel The Love Tonight    1
164水浒全传                          1
The wheels on the bus            1
十年戎马心孤单 - KTV版伴奏                 1
Звезда                           1
The Sounds Of Silence            1
续小八义043                          1
Length: 12334, dtype: int64

In [25]:
df_play.paid_flag.value_counts()

0.0       39583273
0            65467
415.0          427
430.0           32
252.0           26
219.0           19
3.0             13
259.0            6
245.0            6
169.0            6
375.0            4
209.0            4
241.0            4
9.0              3
7.0              3
191.0            3
200.0            3
128.0            2
260.0            2
248.0            2
237.0            2
8.0              1
6.0              1
183.0            1
1026.0           1
211.0            1
235.0            1
725.0            1
爱上一匹野马           1
278.0            1
289.0            1
292.0            1
385.0            1
473.0            1
666.0            1
683.0            1
247.0            1
dtype: int64

In [26]:
df_play[df_play.paid_flag>0]

,uid,device,song_id,song_type,song_name,singer,play_time,song_length,paid_flag,file_name,label
58059,168146144,ar,6916311,0,多少的爱都不要(???? ????????,),Ten Nararak,412,415,0,20170410_2_play.log
58741,168146144,ar,6916311,2,多少的爱都不要(???? ????????,),Ten Nararak,237>}(222.219.141.68)TM,430,0,20170410_2_play.log
164818,168700735,ar,6916311,0,多少的爱都不要(???? ????????,),Ten Nararak,249,415,0,20170410_3_play.log
178183,168647140,ar,6916311,0,多少的爱都不要(???? ????????,),Ten Nararak,258,415,0,20170410_3_play.log
178398,168647140,ar,6916311,0,多少的爱都不要(???? ????????,),Ten Nararak,103,415,0,20170410_3_play.log
433073,0,ar,235500,1,分裂,[内地版,周杰伦,104931,252,0,20170504_3_play.log
447433,168647140,ar,6916311,0,多少的爱都不要(???? ????????,),Ten Nararak,211,415,0,20170504_3_play.log
469348,137084142,ar,6916311,0,多少的爱都不要（???? ????????,）,Ten Nararak,414,415,0,20170504_2_play.log
486671,168271854,ip,6916311,0,多少的爱都不要（???? ????????,）,Ten Nararak,1,415,0,20170504_2_play.log
505531,0,ar,235500,1,分裂,[内地版,周杰伦,104931,252,0,20170504_2_play.log


#### Logs with paid_flag >0 are actually logs with input format errors and paid_flag =0
#### There are no real inputs with paid_flag >0

The important columns for analysis are: song_id, play_time, song_length and file_name so drop the rows with missing values in these three fields.

In [27]:
df_play = df_play.loc[df_play.file_name.notnull() & 
                      df_play.play_time.notnull() & 
                      df_play.song_id.notnull() &
                      df_play.song_length.notnull()]


In [28]:
df_play.isnull().sum(axis = 0)

uid                0
device             0
song_id            0
song_type      12913
song_name        771
singer         37883
play_time          0
song_length        0
paid_flag          0
file_name          0
label              0
dtype: int64

#### Now assign int 0 to the missing values of song_type, which is the most populated song_type

In [29]:
df_play.loc[df_play.song_type.isnull(),'song_type'] = 0
df_play.isnull().sum(axis = 0)

uid                0
device             0
song_id            0
song_type          0
song_name        771
singer         37883
play_time          0
song_length        0
paid_flag          0
file_name          0
label              0
dtype: int64

In [30]:
df_play.shape

(39492892, 11)

In [31]:
# As song_length will be used later, delete logs with >= 0 song_length and null song_name
df_play = df_play.loc[df_play.song_name.notnull() | (df_play.song_length > 0)]
df_play.isnull().sum(axis = 0)

uid                0
device             0
song_id            0
song_type          0
song_name        205
singer         37715
play_time          0
song_length        0
paid_flag          0
file_name          0
label              0
dtype: int64

In [32]:
df_play = df_play.reset_index()

In [33]:
df_play.head(10)

,index,uid,device,song_id,song_type,song_name,singer,play_time,song_length,paid_flag,file_name,label
0,0,168308107,ar,162455,0,最初的梦想,范玮琪,296,296,0,20170410_2_play.log,0
1,1,168112765,ar,4393501,0,喜欢你(f101 粤),Beyond,272,0,0,20170410_2_play.log,0
2,2,168274411,ar,22833011,0,宽恕,宽恕乐队,24,156,0,20170410_2_play.log,0
3,3,0,ar,4266814,1,天使的翅膀,徐誉滕,214384,0,0,20170410_2_play.log,0
4,4,168274411,ar,176292,0,爱不爱我,零点乐队,333,334,0,20170410_2_play.log,0
5,5,168274411,ar,22833011,0,宽恕,宽恕乐队,155,156,0,20170410_2_play.log,0
6,6,168274411,ar,105279,0,曲终人散,张宇,0,0,0,20170410_2_play.log,0
7,7,168274411,ar,176292,0,爱不爱我,零点乐队,0,0,0,20170410_2_play.log,0
8,8,168515688,ip,6586179,0,漂洋过海来看你,孙露,326,326,0,20170410_2_play.log,0
9,9,32166203,ar,1034767,0,好想再爱你,颜亚涛,270,0,0,20170410_2_play.log,0


In [34]:
# add date column
def get_date(file_name):
    tmp_list = str(file_name).split('_')
    return tmp_list[0]

In [35]:
df_play['date'] = df_play['file_name'].map(get_date)

In [36]:
df_play.device.value_counts()

ar           32365780
ip            3826808
ar            2744122
ip             555615
168589573           1
dtype: int64

In [37]:
# Looks like all selected columns using android or iphone.
df_play.device.unique()

array(['ar', 'ip', 'ip ', 'ar ', '168589573'], dtype=object)

In [38]:
def remove_space(word):
    word = str(word).rstrip()
    return word
df_play['device'] = df_play['device'].map(remove_space)

In [39]:
df_play.device.unique()

array(['ar', 'ip', '168589573'], dtype=object)

In [40]:
df_play.head()

,index,uid,device,song_id,song_type,song_name,singer,play_time,song_length,paid_flag,file_name,label,date
0,0,168308107,ar,162455,0,最初的梦想,范玮琪,296,296,0,20170410_2_play.log,0,20170410
1,1,168112765,ar,4393501,0,喜欢你(f101 粤),Beyond,272,0,0,20170410_2_play.log,0,20170410
2,2,168274411,ar,22833011,0,宽恕,宽恕乐队,24,156,0,20170410_2_play.log,0,20170410
3,3,0,ar,4266814,1,天使的翅膀,徐誉滕,214384,0,0,20170410_2_play.log,0,20170410
4,4,168274411,ar,176292,0,爱不爱我,零点乐队,333,334,0,20170410_2_play.log,0,20170410


### save file

In [41]:
df_play.to_csv('/Users/ZhijingYe/Desktop/data/output/play_sample_log.csv',sep='\t')